# Tensorflow for Deep Learning 

Learning Objectives: <br> *By the end of this assignment, you should be familiar using Keras Sequential and Functional APIs for constructing models. You should be comfortable with debugging common modeling errors and researching Tensorflow documentation for various open-ended tasks.*

**Keras** is a deep learning API that runs on top of Tensorflow, with **layers** and **models** as the core data structures. In Tensorflow 2.0, modeling functionalities are under the Keras namespace. (Optional: read about v1 --> v2 API cleanup [here](https://github.com/tensorflow/community/blob/master/rfcs/20180827-api-names.md))

Keras provides a clean, approachable interface with abstractions and building blocks for easy prototyping. 

In [1]:
# tensorflow_version works only in colab
try: 
    %tensorflow_version 2.x
except Exception: 
    pass

import tensorflow as tf
tf.__version__

'2.3.1'

In [2]:
# several examples use the mnist dataset -- hence import & split into trian/valid/test sets
from tensorflow.keras.datasets import mnist
(x_train_full, y_train_full), (x_test,  y_test) = mnist.load_data()
x_train, x_valid = x_train_full[12000:] / 255, x_train_full[:12000] / 255
y_train, y_valid = y_train_full[12000:], y_train_full[:12000]

## 1. Sequential API 

The Sequential API allows one to construct the simplest type of model: one with a linear stock of layers -- model with layers created in a step by step fashion. 

In the examples, we are interested in fashion mnist multi-class classification (ie. models that take 28x28x1 images and classifies each into one of ten classes). 

In [3]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten, Dense

# creates a list of layer definitions
seq_model = Sequential([ 
    # flattens 28x28 image to a 1D array 
    Flatten(input_shape=(28, 28)), 
    # hidden layer with 256 neurons & relu activation
    Dense(256, activation="relu"), 
    # hidden layer with 128 neurons & relu activation 
    Dense(128, activation="relu"), 
    # output layer with 10 neurons & softmax activation 
    Dense(10, activation="softmax")
])

# displays model layers (+ layer (type), output shape, param #)
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


#### 1.1 TO DO: Sequential Questions 
You may consult Tensorflow documentation and online sources for any of the questions below. <br>
a) Describe the architecture of *seq_model* above. (ie. how many/what types of layers? what is a dense? flatten?) <br>
b) What happens if you do not specify an activation function for any one of the dense layers? <br>
c) The three dense layers in *seq_model* have 200960, 32896, and 1290 trainable parameters respectively. Based on your knowledge of densely connected neural networks, explain how these numbers are derived.


After model creation, the next step is to call the *compile()* method and specify a loss function, optimizer, and metrics.

In [14]:
                  # loss -- String (name of objective function), objective function, or Loss instance.
seq_model.compile(loss = "sparse_categorical_crossentropy",
                  # optimizer -- String (name of optimizer) or optimizer instance. 
                  optimizer = "sgd", 
                  # list of metrics to be evaluated by the model during training and testing.
                  # each can be a String (name of built-in function), function, or Metric instance. 
                  metrics = ["accuracy"])

Finally, we can train the model by calling the *fit()* function. Notice that calling the *fit()* function returns a *History* object, with its *History.history* attribute recording training loss and metrics values at every epoch. 

In [15]:
                            # input data
seq_history = seq_model.fit(x = x_train, 
                            # input labels
                            y = y_train, 
                            # epoch -- an iteration over the entire x and y dataset provided
                            epochs = 5, 
                            # data on which to evaluate the loss and any model metrics at the ennd of each epoch
                            validation_data = (x_valid, y_valid))

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1707 - accuracy: 0.9519 - val_loss: 0.1733 - val_accuracy: 0.9495
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1541 - accuracy: 0.9561 - val_loss: 0.1608 - val_accuracy: 0.9527
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1393 - accuracy: 0.9609 - val_loss: 0.1514 - val_accuracy: 0.9553
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1273 - accuracy: 0.9638 - val_loss: 0.1375 - val_accuracy: 0.9609
Epoch 5/5
1500/1500 [==============================] - 1s 982us/step - loss: 0.1165 - accuracy: 0.9671 - val_loss: 0.1308 - val_accuracy: 0.9639


While the Sequential API is easy to use, we cannot create models that share layers, have branches, nor have multiple inputs/outputs. However, we *can* with the Functional API. 

## 2. Functional API

When constructing models with the Functional API, follow three important steps: <br>
1) explicitly define the input layer <br> 
2) define model layers, connecting each layer using Python functional syntax <br>
3) define the model by callling the model object and giving it the input and output layers

*func_model* below contains the same architecture as *seq_model*, but uses the Functional API. 

In [21]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model 

# define input tensor
input_layer = Input(shape = (28, 28))

# stack layers using the syntax: current_layer()(previous_layer)
flattened = Flatten()(input_layer)
fc1 = Dense(256, activation = "relu")(flattened)
fc2 = Dense(128, activation = "relu")(fc1)
predictions = Dense(10, activation = "softmax")(fc2)

# define model object -- specify input and outputs
func_model = Model(inputs = [input_layer], outputs = [predictions])

# displays model layers (+ layer (type), output shape, param #)
func_model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


As with the Sequential model, we call the *compile()* method and specify a loss function, optimizer, and metrics. Then, call the *fit()* function and plot metrics.  

In [22]:
func_model.compile(loss = "sparse_categorical_crossentropy", 
                   optimizer = "sgd", 
                   metrics = ["accuracy"])

func_history = func_model.fit(x_train, 
                              y_train, 
                              epochs = 5, 
                              validation_data = (x_valid, y_valid))

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6813 - accuracy: 0.8241 - val_loss: 0.3372 - val_accuracy: 0.9069
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3124 - accuracy: 0.9118 - val_loss: 0.2747 - val_accuracy: 0.9234
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2581 - accuracy: 0.9260 - val_loss: 0.2403 - val_accuracy: 0.9327
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2229 - accuracy: 0.9373 - val_loss: 0.2108 - val_accuracy: 0.9398
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1962 - accuracy: 0.9445 - val_loss: 0.1893 - val_accuracy: 0.9455


#### 2.1 TO DO: Functional Questions 
You may consult Tensorflow documentation and online sources for the questions below. <br>
a) What is meant by *functional syntax*? <br>
b) What are the advantages of using the functional syntax? <br>

#### 2.2 TO DO: Plot Learning Curves
Refit any of the two models above, but with epochs = 20. Then evaluate the corresponding metrics in the following cell and plot two graphs: <br>
a. train & validation loss for every epoch <br>
b. train & validation accuracy for every epoch
            
You may import any necessary libraries. 

In [20]:
seq_metrics = seq_history.history
func_metrics = func_history.history

print(type(seq_metrics), type(func_metrics))

### your code here ###

<class 'dict'> <class 'dict'>


In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Run cell -- you should see an image of a flower
from sklearn.datasets import load_sample_image

flower = load_sample_image('flower.jpg')
    
_ = plt.imshow(flower)

In [ ]:
# Run cell -- dataset of one 100 x 100 x 3 RGB image
dataset = np.array([flower], dtype = np.float32)

count, height, width, channels = dataset.shape
count, height, width, channels

In [ ]:
# *** TO DO ***
# 1) Convert dataset to tensor object
data = pass

In [ ]:
# *** TO DO ***
# 2) Create a vertical line filter that will be applied using Tensorflow's tf.nn.conv2d function
filter_height, filter_width = 10, 10
channels_input, channels_output = 3, 1

filters = np.zeros(shape = (pass, pass, pass, pass), dtype = np.float32)
filters[:, 5:7, :, 0] = 1

In [ ]:
# *** TO DO ***
# 3) create a convolution layer using tf.nn.conv2d and pass in the following arguments: image we are processing, filters, and strides
convolution = ...(..., ..., strides = [1, 10, 10, 1], padding = "SAME")
output = convolution.numpy()

In [ ]:
# Run cell 
plt.imshow(output[0, :, : 0], cmap = "gray")
plt.show()

In [ ]:
# *** To DO *** 
# 4) Using tf.keras.layers.Conv2D, create a layer with three filters, a 5x5 
# visual receptor, and stride of 2
convolution = ......(filters = pass, kernel_size = pass, strides = pass, padding = "SAME")
output = convolution(X).numpy()

In [ ]:
# *** TO DO ***
# 5) Plot the first feature map: 
plt.imshow(output[pass, :, :, pass])
plt.show()

In [ ]:
# *** To DO ***
# 6) Plot the second feature map: 
plt.imshow(output[pass, :, :, pass])
plt.show()

In [ ]:
# *** TO DO *** 
# 7) Plot the third feature map: 
plt.imshow(ouutput[pass, :, : pass])
plt.show()

In [ ]:
# *** TO DO *** 
# 8) Pass the flower image into the tf.nn.max_pool function below
size = [1, 2, 2, 1]
maxpool = tf.nn.maxpool(pass, ksize = size, strides = size, padding = "VALID")
output = maxpool.numpy()
plt.imshow(output[0].astype(np.uint8))
plt.show()

## Final Exercise
Let's build a model that classifies images into 10 different categories. 
Requirements: 3 convolution layers, each followed by a maxpooling layer; 2 dense layers at the end; adam optimization

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers. ...(16, 3, padding='same', activation= pass,
                           input_shape=(100, 100, 3)),
    tf.keras.layers. ...,
    tf.keras.layers. ... (32, 3, padding='same', activation= pass),
    tf.keras.layers. ...,
    tf.keras.layers. ... (64, 3, padding='same', activation= pass),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers. ... (256, activation= pass),
    tf.keras.layers. ... (128, activation= pass)
])

model.compile(optimizer = pass
              loss = pass,
              metrics = ['accuracy'])

model.summary()